## Optical Character Recognition of Maps using the MapReader Library
<p>By: Ben Woodward</p>
<p>August 6th, 2024</p>
<p>(Intro text to be added...)</p>
<p>To learn more about the MapReader project, click <a href=''>here</a>.

### Install required libraries
Use PIP to create a virtual environment and install the libraries used in this script.

In [ ]:
#Install dependencies
#Install MapReader
#Install detectron2
#Install DeepSolo, DPText, and MapTextPipeline

### Decrease Map Resolution
To decrease compute time, this code takes your original map scan and reduces its resolution to 96 DPI.

In [ ]:
#Specify your preferred resolution, in DPI, here.
resolution = 96

from PIL import Image

# Open the image file
image_path = 'belden_map_bayfield.jpg'
img = Image.open(image_path)

# Decrease resolution to your desired DPI level
new_dpi = (resolution, resolution)

# Save the image with new DPI
output_path = image_path.split('.')[0] + resolution + 'dpi' + image_path.split('.')[1]
img.save(output_path, dpi=new_dpi)

print(f"Image saved at {output_path} with DPI {new_dpi}")

### Georeference your map
Before proceeding to the next step, georeference your map in ArcGIS Pro or QGIS. This code is set up for georeferencing in EPSG: 4326.

### Load your map
This code loads your map into the mapreader library for processing.

In [1]:
#Specify the path to your georeferenced map here
georeferenced_map_filename = "belden_map_bayfield_96dpi_georef_4326.tif"

from mapreader import loader
my_files = loader(georeferenced_map_filename)
my_files.add_geo_info()

  0%|          | 0/1 [00:00<?, ?it/s]

### Set the algorithm parameters
Try changing these parameters to improve the accuracy of the algorithm's predictions for your map.

In [2]:
patch_size = 1024 #The documentation recommends using patches that are 1024x1024 pixels
patch_overlap = 0.1 #Determine to what degree each patch overlaps with its neighbours. Experiment with different values to see what works best
min_text_overlap = 0.7 #Determine the threshold for grouping nearby letters and words into one record

### Divide your map into patches
Many ML workflows involve dividing the image into patches/tiles to speed up computing times.

In [3]:
my_files.patchify_all(path_save="./patches_1024", patch_size=patch_size, overlap=patch_overlap, rewrite=True)
my_files.save_patches_as_geotiffs()

[INFO] Saving patches in directory named "./patches_1024".


  0%|          | 0/1 [00:00<?, ?it/s]

belden_map_bayfield_96dpi_georef_4326.tif


  0%|          | 0/42 [00:00<?, ?it/s]

### Run the DeepSolo ML Algorithm on your Map

The DeepSolo algorithm can be used to detect text on your maps. This code block runs the algorithm. The algorithm can take several minutes to hours to run depending on the resolution and size of your map.

If the algorithm is taking too long to run, you can try decreasing the resolution of your scanned map using the first code block. However, this may reduce the accuracy of the algorithm's output. That said, I have found that 96 DPI is sufficient in most cases.

The MapReader library supports other algorithms, including DPText-DETR and MapTextPipeline. More information on those algorithms can be found <a href='https://mapreader.readthedocs.io/en/latest/using-mapreader/step-by-step-guide/6-spot-text.html'>here</a>.

The parameters of the DeepSolo algorithm can be adjusted by modifying the "finetune_150k_tt_mlt_13_15_textocr.yaml" file. This is beyond the scope of this tutorial.

In [5]:
# from mapreader.spot_text.deepsolo_runner import DeepSoloRunner
from mapreader.spot_text.maptext_runner import MapTextRunner
# from mapreader.spot_text.dptext_detr_runner import DPTextDETRRunner

parent_df, patch_df = my_files.convert_images(save=True)

#This one is the best balance between predictive power and performance. It can be run on a typical laptop.
# deepsolo_runner = DeepSoloRunner(
#     patch_df,
#     parent_df,
#     cfg_file = "./DeepSolo/configs/R_50/IC15/finetune_150k_tt_mlt_13_15_textocr.yaml",
#     weights_file = "./weights/ic15_res50_finetune_synth-tt-mlt-13-15-textocr.pth"
# )

#This one shows you where it thinks the text is on the map, but does not digitize the text. It can be run on a typical laptop.
# dptext_runner = DPTextDETRRunner(
#     patch_df,
#     parent_df,
#     cfg_file = "DPText-DETR/configs/DPText_DETR/ArT/R_50_poly.yaml",
#     weights_file = "./weights/art_final.pth",
# )

#This one likely has the best predictive power, but is very computationally intensive.
maptext_runner = MapTextRunner(
    "./patch_df.csv",
    "./parent_df.csv",
    cfg_file = "MapTextPipeline/configs/ViTAEv2_S/rumsey/final_rumsey.yaml",
    weights_file = "./weights/rumsey-finetune.pth"
)

my_runner = maptext_runner

patch_preds_df = my_runner.run_all(return_dataframe=True, min_ioa=min_text_overlap)

[INFO] Saved parent dataframe as "parent_df.csv"
[INFO] Saved patch dataframe as "patch_df.csv"
[INFO] Reading "./patch_df.csv".
[INFO] Reading "./parent_df.csv".


  0%|          | 0/42 [00:00<?, ?it/s]

### Save the output as a GeoJSON
Save the text predictions as a GeoJSON file that can be visualized in GIS software. The MapReader library saves the predictions as polygons deliniating the extent of the text it read with its prediction for the text as an attribute ("text"). There is another attribute ("score") that denotes how confident the algorithm is in its prediction.

In [ ]:
parent_preds_df = my_runner.convert_to_parent_pixel_bounds(return_dataframe=True, deduplicate=True, min_ioa=min_text_overlap)
geo_preds_df = my_runner.convert_to_coords(return_dataframe=True)
my_runner.save_to_geojson("text_preds_dptext.geojson")
# geo_preds_df.to_csv("geo_preds.csv")